In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import os
import json
os.chdir('/home/composersyf/Documents/Google Civic Information API/Second_Batch')

In [2]:
with open("final_result_mail_round2_cleaned.json") as json_file_1:
    json_file_1=json_file_1.readlines()
    #json_data_1=str(json_file_1)
    #print(type(str(json_file_1)))
    #json_file_1[909609]='{"47519":{}},\n'
    #json_data_1=''
    #for l in json_file_1:
        #json_data_1+=l
    #json_data_2=json_data_1[:7395701]+"{}"+json_data_1[7395701:]
    json_data_2=json.loads(json_file_1[0])
len(json_data_2)

17740

In [3]:
#transform query result list to dictionary
json_data_3={}
for d in json_data_2:
    k=list(d.keys())[0]
    json_data_3[k]=d[k]

In [4]:
len(json_data_3)

17740

In [5]:
query_info={}
for d in json_data_2:
    k=list(d.keys())[0]
    try:
        query_info[k]={'status':'has_polling_info','Polling_Location_Info':d[k][0]['pollingLocations'][0]}
    except KeyError:
        try:
            query_info[k]={'status':'early_voting','EV_info':d[k][0]['earlyVoteSites']}
        except KeyError:
            try:
                a=d[k][0]['normalizedInput']
                query_info[k]={'status':'no_polling_info'}
            except KeyError:
                query_info[k]={'status':'recheck'}

In [6]:
#check individual query results
#query_info['377408']

In [7]:
query_info_status=[(k,query_info[k]['status']) for k in query_info]
query_info_status_2=[query_info[k]['status'] for k in query_info]
Counter(query_info_status_2)

Counter({'early_voting': 6240,
         'has_polling_info': 3754,
         'no_polling_info': 7310,
         'recheck': 436})

In [8]:
recheck_list=[]
for s in query_info_status:
    if s[1]=="recheck":
        recheck_list.append(s[0])

In [9]:
error_message={}
for i in recheck_list:
    #print(json_data_3[i][0]['error']['message'])
    error_message[i]=json_data_3[i][0]['error']['message']
Counter(error_message.values())

Counter({'Failed to parse address': 213,
         'No information for this address': 223})

In [10]:
#find the index of addresses which end up with Backend Error
'''
back_end_error_list=[]
for i in error_message:
    if error_message[i]=='Backend Error':
        back_end_error_list.append(i)
back_end_error_list
'''

"\nback_end_error_list=[]\nfor i in error_message:\n    if error_message[i]=='Backend Error':\n        back_end_error_list.append(i)\nback_end_error_list\n"

In [12]:
#change the status from 'recheck' to 'error' 
for i in error_message:
    #if i not in back_end_error_list:
    query_info[i]['status']='error'
    query_info[i]['error_message']=error_message[i]

In [41]:
#Correct the Backend Errors
'''
ev_info='''[
  {
   "address": {
    "line1": "1425 East High Street",
    "city": "Bryan",
    "state": "OH",
    "zip": "43506"
   },
   "notes": "Ste 104",
   "pollingHours": "Tue, Oct 25: 8 am - 6 pm\nWed, Oct 26: 8 am - 6 pm\nThu, Oct 27: 8 am - 6 pm\nFri, Oct 28: 8 am - 6 pm\nSat, Oct 29: 8 am - 4 pm\nSun, Oct 30: 1 pm - 5 pm\nMon, Oct 31: 8 am - 7 pm\nTue, Nov 1: 8 am - 7 pm\nWed, Nov 2: 8 am - 7 pm\nThu, Nov 3: 8 am - 7 pm\nFri, Nov 4: 8 am - 7 pm\nSat, Nov 5: 8 am - 4 pm\nSun, Nov 6: 1 pm - 5 pm\nMon, Nov 7: 8 am - 2 pm",
   "startDate": "2016-10-12",
   "endDate": "2016-11-07",
   "sources": [
    {
     "name": "DemocracyWorks",
     "official": false
    }
   ]
  }
 ]'''
'''

In [46]:
#query_info['408954']['status']='early_voting'
#query_info['408954']['EV_info']=ev_info

In [13]:
#modify query_info_status after 'recheck' was changed:
query_info_status=[(k,query_info[k]['status']) for k in query_info]
query_info_status_2=[query_info[k]['status'] for k in query_info]
Counter(query_info_status_2)

Counter({'early_voting': 6240,
         'error': 436,
         'has_polling_info': 3754,
         'no_polling_info': 7310})

In [14]:
#good_list: contains the addresses that actually return pollingLocations
good_list=[]
for s in query_info_status:
    if s[1]=="has_polling_info":
        good_list.append(s[0])
good_address=pd.DataFrame({'column1':good_list})
good_address.to_csv("good_address_text.csv",index=False,header=False)

In [16]:
print(good_list[:20])

['103530', '126415', '23892', '119049', '123130', '207146', '6534', '8451', '37424', '8595', '128720', '1844', '14868', '208186', '20375', '214138', '19251', '110998', '6166', '207862']


#### Dealing with good_list:

In [17]:
data_dict_good={}
for i in good_list:
    data_dict_good[i]={}
    try:
        address_info=query_info[i]['Polling_Location_Info']
        try:
            data_dict_good[i]['city']=address_info['address']['city']
        except KeyError:
            pass
        try:
            data_dict_good[i]['line1']=address_info['address']['line1']
        except KeyError:
            pass
        try:
            data_dict_good[i]['line2']=address_info['address']['line2']
        except KeyError:
            pass
        try:
            data_dict_good[i]['line3']=address_info['address']['line3']
        except KeyError:
            pass
        try:
            data_dict_good[i]['locationName']=address_info['address']['locationName']
        except KeyError:
            pass
        try:
            data_dict_good[i]['pollingHours']=address_info['pollingHours']
        except KeyError:
            pass
    except (IndexError,KeyError):
        pass

In [19]:
#data_dict_good['207862']

In [20]:
data_df_good=pd.DataFrame.from_dict(data_dict_good,orient="index")
data_df_good=data_df_good.reset_index([0]).rename(columns={'index':'voter_id'})
data_df_good.voter_id=data_df_good.voter_id.astype(int)
data_df_good=data_df_good.sort_values(by=['voter_id']).fillna("")
data_df_good.head()

,voter_id,locationName,pollingHours,city,line1,line2,line3
3647,79,NEWTOWN GRANT HOA REC CENTER,7 am - 8 pm,NEWTOWN,360 EAGLE RD,,
1401,133,GRACEDALE NURSING HOME,7 am - 8 pm,NAZARETH,2 GRACEDALE AVE,,
1488,152,FOREST GROVE PRESBYTERIAN CHURCH,7 am - 8 pm,FOREST GROVE,1856 FOREST GROVE RD,,
1575,169,MONITEAU HIGH SCHOOL (MEETING ROOM),7 am - 8 pm,WEST SUNBURY,1810 WEST SUNBURY RD,,
1582,170,ROBERTS ELEMENTARY SCHOOL,7 am - 8 pm,WAYNE,889 CROTON RD,,


In [21]:
data_df_good['pollingAddress']=data_df_good.line1+', '+data_df_good.line2+', '+data_df_good.line3
data_df_good['pollingAddress']=data_df_good.pollingAddress.apply(str.strip).apply(str.rstrip,args=(',',)).apply(str.strip).apply(str.rstrip,args=(',',)).apply(str.strip)
data_df_good=data_df_good.drop(["line1","line2","line3"],axis=1).rename(columns={'locationName':"pollingLocationName"})
data_df_good.head()

,voter_id,pollingLocationName,pollingHours,city,pollingAddress
3647,79,NEWTOWN GRANT HOA REC CENTER,7 am - 8 pm,NEWTOWN,360 EAGLE RD
1401,133,GRACEDALE NURSING HOME,7 am - 8 pm,NAZARETH,2 GRACEDALE AVE
1488,152,FOREST GROVE PRESBYTERIAN CHURCH,7 am - 8 pm,FOREST GROVE,1856 FOREST GROVE RD
1575,169,MONITEAU HIGH SCHOOL (MEETING ROOM),7 am - 8 pm,WEST SUNBURY,1810 WEST SUNBURY RD
1582,170,ROBERTS ELEMENTARY SCHOOL,7 am - 8 pm,WAYNE,889 CROTON RD


In [22]:
my_df_good=pd.read_csv("batch2_mail.csv",header=None,sep='\t').merge(data_df_good,how="inner",left_on=[0],right_on=['voter_id']).iloc[:,[0,1,3,4,5,6]].rename(columns={1:"homeAddress",'city':"pollingCity"})
my_df_good['has_polling_info']=True
my_df_good.head()

,0,homeAddress,pollingLocationName,pollingHours,pollingCity,pollingAddress,has_polling_info
0,79,"58 Brookdale Pl , Newtown, PA, 18940",NEWTOWN GRANT HOA REC CENTER,7 am - 8 pm,NEWTOWN,360 EAGLE RD,True
1,133,"3206 Silver Crest Rd , Nazareth, PA, 18064",GRACEDALE NURSING HOME,7 am - 8 pm,NAZARETH,2 GRACEDALE AVE,True
2,152,"2327 Dorchester St , Furlong, PA, 18925",FOREST GROVE PRESBYTERIAN CHURCH,7 am - 8 pm,FOREST GROVE,1856 FOREST GROVE RD,True
3,169,"400 Grubb Rd , Boyers, PA, 16020",MONITEAU HIGH SCHOOL (MEETING ROOM),7 am - 8 pm,WEST SUNBURY,1810 WEST SUNBURY RD,True
4,170,"601 Bob White Rd , Wayne, PA, 19087",ROBERTS ELEMENTARY SCHOOL,7 am - 8 pm,WAYNE,889 CROTON RD,True


In [23]:
print(my_df_good.shape)
#print(good_list[:20])

(3754, 7)


In [24]:
def is_element(e,l):
    if e in l:
        return True
    else:
        False

In [25]:
original=pd.read_csv("civicnation_address_gotv_mail_clean.csv",low_memory=False).reset_index([0])
my_df_good_final=original.merge(my_df_good,how="inner",left_on=['index'],right_on=[0]).drop(['index','homeAddress'],axis=1)
my_df_good_final['data_available']=True
my_df_good_final=my_df_good_final.set_index([0])
my_df_good_final

,customer_id,address,addr1,addr2,city,state,zip_code,pollingLocationName,pollingHours,pollingCity,pollingAddress,has_polling_info,data_available
0,,,,,,,,,,,,,
79,ai_500317,"58 Brookdale Pl , Newtown, PA, 18940",58 Brookdale Pl,NaN,Newtown,PA,"18,940",NEWTOWN GRANT HOA REC CENTER,7 am - 8 pm,NEWTOWN,360 EAGLE RD,True,True
133,ai_500533,"3206 Silver Crest Rd , Nazareth, PA, 18064",3206 Silver Crest Rd,NaN,Nazareth,PA,"18,064",GRACEDALE NURSING HOME,7 am - 8 pm,NAZARETH,2 GRACEDALE AVE,True,True
152,ai_500609,"2327 Dorchester St , Furlong, PA, 18925",2327 Dorchester St,NaN,Furlong,PA,"18,925",FOREST GROVE PRESBYTERIAN CHURCH,7 am - 8 pm,FOREST GROVE,1856 FOREST GROVE RD,True,True
169,ai_500677,"400 Grubb Rd , Boyers, PA, 16020",400 Grubb Rd,NaN,Boyers,PA,"16,020",MONITEAU HIGH SCHOOL (MEETING ROOM),7 am - 8 pm,WEST SUNBURY,1810 WEST SUNBURY RD,True,True
170,ai_500681,"601 Bob White Rd , Wayne, PA, 19087",601 Bob White Rd,NaN,Wayne,PA,"19,087",ROBERTS ELEMENTARY SCHOOL,7 am - 8 pm,WAYNE,889 CROTON RD,True,True
219,ai_500078,"636 Georges Ln , Ardmore, PA, 19003",636 Georges Ln,NaN,Ardmore,PA,"19,003",CHESTNUTWOLD ELEMENTARY SCHOOL,7 am - 8 pm,ARDMORE,630 LORAINE ST,True,True
285,ai_500075,"508 Beltzhoover Ave , Pittsburgh, PA, 15210",508 Beltzhoover Ave,NaN,Pittsburgh,PA,"15,210",MCKINLEY PARK REC CENTER,7 am - 8 pm,PITTSBURGH,900 DESDEMONT AVE,True,True
297,ai_500123,"134 Plymouth Rd , Plymouth Meeting, PA, 19462",134 Plymouth Rd,NaN,Plymouth Meeting,PA,"19,462",CHURCH ON THE MALL,7 am - 8 pm,PLYMOUTH MTG,"500 W GERMANTOWN PIKE , PLYMOUTH MEETING MALL",True,True
310,ai_500175,"647 Dayton Rd , Bryn Mawr, PA, 19010",647 Dayton Rd,NaN,Bryn Mawr,PA,"19,010",HAVERFORD MONTHLY MTNG OF FRIENDS,7 am - 8 pm,HAVERFORD,855 BUCK LN,True,True


In [26]:
sum(np.array(my_df_good_final.index)==sorted(np.array(good_list).astype(int)))

3754

In [28]:
mail_v2=pd.read_csv("batch2_mail_results_v1.csv",low_memory=False)
mail_v2_part1=pd.DataFrame.copy(my_df_good_final)
mail_v2_part2=mail_v2.iloc[np.setdiff1d(np.array(mail_v2.index),np.array(good_list)),:]
mail_v3=pd.concat([mail_v2_part1,mail_v2_part2],join="outer",axis=0).sort_index()

In [29]:
len(np.setdiff1d(np.array(mail_v2.index),np.array(good_list)))+len(good_list)

259639

In [30]:
mail_v3.to_csv("batch2_mail_results_v3.csv",index=False)
#text_v2.iloc[249766,:]

### The Last Step!!!

In [31]:
remove_list=[]
for i in query_info:
    if query_info[i]['status']=='has_polling_info' or query_info[i]['status']=='recheck':
        remove_list.append(i)
print(len(query_info))
for i in remove_list:
    del query_info[i]
print(len(query_info))

17740
13986


In [32]:
second_round=pd.DataFrame.from_dict(query_info,orient='index')
second_round.head()

,EV_info,status,error_message
100001,NaN,no_polling_info,NaN
100013,NaN,no_polling_info,NaN
100014,NaN,no_polling_info,NaN
100029,NaN,no_polling_info,NaN
100061,"[{'address': {'city': 'West Covina', 'zip': '9...",early_voting,NaN


In [33]:
second_round.shape

(13986, 3)

In [100]:
#second_round.loc['1126','status']='early_voting'
#second_round.loc['1126','error_message']=''
#second_round.loc['1126','EV_info']=query_info_2['0']['EV_info']

In [101]:
#second_round.loc['1618','status']='early_voting'
#second_round.loc['1618','error_message']=''
#second_round.loc['1618','EV_info']=query_info_2['1']['EV_info']

In [102]:
#second_round.loc['1618',:]

In [34]:
second_round=second_round.reset_index([0])
second_round.iloc[:,0]=second_round.iloc[:,0].apply(int)
second_round=second_round.rename(columns={'index':'voter_id'}).sort_values(['voter_id'])
second_round.shape

(13986, 4)

In [36]:
second_round.head()

,voter_id,EV_info,status,error_message
11495,59,"[{'address': {'city': 'Rockford', 'zip': '6110...",early_voting,NaN
11844,65,"[{'address': {'city': 'Oakland', 'zip': '94612...",early_voting,NaN
12576,77,NaN,no_polling_info,NaN
13637,94,NaN,no_polling_info,NaN
13819,97,NaN,no_polling_info,NaN


In [37]:
final_data=mail_v3.reset_index([0]).merge(second_round,how="outer",left_on=['index'],right_on="voter_id").drop(['voter_id','index'],axis=1)

In [38]:
column_order=['customer_id','address','addr1','addr2','city','state','zip_code','data_available',
             'has_polling_info','pollingLocationName','pollingCity','pollingHours','pollingAddress',
             'status','EV_info','error_message']
mail_v4=final_data.loc[:,column_order]
mail_v4.to_csv("batch2_mail_results_v4.csv",index=False)

In [39]:
mail_v4.shape

(259639, 16)

### Completed!!!

In [40]:
os.chdir("/home/composersyf/Documents/Google Civic Information API/Deliverables")

In [47]:
mail_v0=pd.read_csv('../Second_Batch/civicnation_address_gotv_mail_clean.csv',low_memory=False)
mail_v0.shape

(259639, 7)

In [42]:
deliverable_mail=pd.read_csv('batch2_mail_results_version1.csv',low_memory=False)

In [44]:
len(np.array(deliverable_mail.index))

259639

In [48]:
deliverable_mail

,customer_id,address,addr1,addr2,city,state,zip_code,data_available,has_polling_info,pollingLocationName,pollingCity,pollingHours,pollingAddress,status,EV_info,error_message
0,ai_500001,"2216 38th St , Minneapolis, MN, 55407",2216 38th St,NaN,Minneapolis,MN,"55,407",True,True,ROOSEVELT HIGH SCHOOL (GYM),MINNEAPOLIS,7 am - 8 pm,4029 28TH AVE S,NaN,NaN,NaN
1,ai_500005,"2105 Fall Hill Ave , Fredericksburg, VA, 22401",2105 Fall Hill Ave,NaN,Fredericksburg,VA,"22,401",True,True,DOROTHY HART COMMUNITY CENTER,FREDERICKSBURG,6:00 AM - 7:00 PM,408 CANAL STREET,NaN,NaN,NaN
2,ai_500009,"330 De Neve Dr , Los Angeles, CA, 90024",330 De Neve Dr,NaN,Los Angeles,CA,"90,024",True,True,UCLA REIBER HALL,LOS ANGELES,7:00am-8:00pm,310 DE NEVE DR,NaN,NaN,NaN
3,ai_500013,"24 Birch Rd , Staten Island, NY, 10303",24 Birch Rd,NaN,Staten Island,NY,"10,303",True,True,Staten Island School of Civic Leadership,Staten Island,6 am - 9 pm,280 Regis Drive,NaN,NaN,NaN
4,ai_500017,"910 Washburn Ave , Minneapolis, MN, 55411",910 Washburn Ave,NaN,Minneapolis,MN,"55,411",True,True,NORTH POINT HEALTH/WELLNESS CTR INC,MINNEAPOLIS,7 am - 8 pm,1315 PENN AVE N,NaN,NaN,NaN
5,ai_500021,"3390 Hillside Garden Dr , Las Vegas, NV, 89135",3390 Hillside Garden Dr,NaN,Las Vegas,NV,"89,135",True,True,"GOOLSBY, JUDY AND JOHN ELEMENTARY",LAS VEGAS,7 am - 7 pm,11175 W DESERT INN RD,NaN,NaN,NaN
6,ai_500025,"460 Heritage Rd , Southbury, CT, 6488",460 Heritage Rd,NaN,Southbury,CT,"6,488",True,True,Southbury Public Library District #2,Southbury,6 am - 8 pm,100 Poverty Road,NaN,NaN,NaN
7,ai_500029,"2152 Donald Rd , Bethel, OH, 45106",2152 Donald Rd,NaN,Bethel,OH,"45,106",True,True,BETHEL-TATE MIDDLE SCHOOL,BETHEL,6:30 am - 7:30 pm,649 W PLANE ST,NaN,NaN,NaN
8,ai_500033,"6521 Rockland Dr , Clifton, VA, 20124",6521 Rockland Dr,NaN,Clifton,VA,"20,124",True,True,CENTREVILLE HIGH SCHOOL,CLIFTON,6:00 AM - 7:00 PM,6001 UNION MILL RD,NaN,NaN,NaN
9,ai_500037,"916 1st St , Moore, OK, 73160",916 1st St,NaN,Moore,OK,"73,160",True,True,ST. ANDREWS CATHOLIC CHURCH,MOORE,7:00 AM - 7:00 PM,800 NW 5 ST,NaN,NaN,NaN


In [87]:
sum(~pd.isnull(deliverable_mail.error_message))

436

In [92]:
Counter(deliverable_mail.has_polling_info)

Counter({nan: 55944, False: 13986, True: 189709})

In [79]:
second_round_index=pd.read_csv("batch1_late_second_round_index.csv",header=None).reset_index([0]).drop([1],axis=1)

In [80]:
second_round_result=pd.concat([second_round_index,second_round_1],join="inner",axis=1).iloc[:,[1,3,4,5]].rename(columns={0:"old_index"})

In [81]:
second_round_result=second_round_result.iloc[:,[0,2,1,3]]

In [82]:
batch1_late_v1=pd.read_csv("batch1_late_v1.csv",low_memory=False)

In [83]:
batch1_late_v2=batch1_late_v1.reset_index([0]).merge(second_round_result,how="outer",left_on=['index'],right_on=["old_index"]).drop(['index','old_index'],axis=1)

In [84]:
batch1_late_v2.to_csv("batch1_late_v2.csv",header=True,index=False)

In [91]:
batch1_late_v2[batch1_late_v2.status=='error'].shape

(411, 17)

In [166]:
batch1_early_v2.iloc[15566,:].pollingHours

'Open the following hours:\nFriday, Nov. 4, 10 a.m. - 5 p.m. \nSaturday, Nov. 5, 10 a.m. – 3 p.m. \nMonday, Nov. 7, 10 a.m. - 7 p.m. \nElection day, Nov. 8, 10 a.m. - 8 p.m.'

In [18]:
data_dict_1={}
for e in json_data_2:
    k=list(e.keys())[0]
    data_dict_1[k]={}
    try:
        address_info=e[k][0]['pollingLocations'][0]
        #address_info=e[k]['pollingLocations'][0]
        try:
            data_dict_1[k]['locationName']=address_info['address']['locationName']
            data_dict_1[k]['city']=address_info['address']['city']
            data_dict_1[k]['line1']=address_info['address']['line1']
            data_dict_1[k]['line2']=address_info['address']['line2']
            data_dict_1[k]['line3']=address_info['address']['line3']
        except KeyError:
            pass
        try:
            data_dict_1[k]['pollingHours']=address_info['pollingHours']
        except KeyError:
            pass
    except KeyError:
        pass

In [6]:
list(json_data_1[0].keys())[0]

'1'

In [11]:
e[k][0]

{'pollingLocations': [{'address': {'city': 'Bellingham',
    'line1': '311 Grand Ave',
    'locationName': "Whatcom County Auditor's Office",
    'state': 'WA',
    'zip': '98225'},
   'notes': '',
   'pollingHours': 'Open: 8:30 a.m. - 4:30 p.m., Monday - Friday\nElection Day Only: 8:30 a.m. - 8:00 p.m.',
   'sources': [{'name': 'Voting Information Project', 'official': True}]}]}

In [19]:
import pandas as pd
import numpy as np

In [23]:
data_df_1=pd.DataFrame.from_dict(data_dict_1,orient="index")
data_df_1=data_df_1.reset_index([0]).rename(columns={'index':'voter_id'})
data_df_1.voter_id=data_df_1.voter_id.astype(int)
data_df_1=data_df_1.sort_values(by=['voter_id']).fillna("")
data_df_1

,voter_id,city,locationName,line1,pollingHours,line2,line3
0,0,WARREN,OUR LADY OF THE MOUNT PARISH CENTER,167 MOUNT BETHEL RD,6:00am-8:00pm,,
1,1,ORADELL,BOROUGH HALL MULTI PURPOSE RM.2ND FL.,355 KINDERKAMACK RD,6:00am-8:00pm,,
10337,2,ORANGE,SALEM TOWERS (H),98 HIGH ST,6:00am-8:00pm,SOCIAL ROOM,
20616,3,MADISON,MADISON YMCA FRONT LEFT LOBBY,111 KINGS RD,6:00am-8:00pm,,
30858,4,MILLBURN,DEERFIELD SCHOOL (H),26 TROY LN,6:00am-8:00pm,,
51299,6,KINNELON BOROUGH,FAYSON LAKES CLUBHOUSE LARGE ROOM,10 CLUBHOUSE TR,6:00am-8:00pm,,
55169,7,JERSEY CITY,PS 6 JOTHAM W WAKEMAN SCHOOL,100 SAINT PAULS AVE,6:00am-8:00pm,,
56237,8,ROCKAWAY TOWNSHIP,DENNIS B O'BRIEN SCHOOL GYM,418 MINERAL SPRING DR,6:00am-8:00pm,,
57287,9,TENAFLY,PRESBYTERIAN CHURCH AUDITORIUM,55 MAGNOLIA AVE,6:00am-8:00pm,,
1051,11,MONTCLAIR,BRADFORD SCHOOL (H),87 MOUNT HEBRON RD,6:00am-8:00pm,(GYM STAGE SIDE),


In [24]:
data_df_1['pollingAddress']=data_df_1.line1+', '+data_df_1.line2+', '+data_df_1.line3
data_df_1['pollingAddress']=data_df_1.pollingAddress.apply(str.strip).apply(str.rstrip,args=(',',)).apply(str.strip).apply(str.rstrip,args=(',',)).apply(str.strip)

In [25]:
data_df_1=data_df_1.drop(["line1","line2","line3"],axis=1).rename(columns={'locationName':"pollingLocationName"})
#data_df_1=data_df_1.drop(["line3"],axis=1)
data_df_1

,voter_id,city,pollingLocationName,pollingHours,pollingAddress
0,0,WARREN,OUR LADY OF THE MOUNT PARISH CENTER,6:00am-8:00pm,167 MOUNT BETHEL RD
1,1,ORADELL,BOROUGH HALL MULTI PURPOSE RM.2ND FL.,6:00am-8:00pm,355 KINDERKAMACK RD
10337,2,ORANGE,SALEM TOWERS (H),6:00am-8:00pm,"98 HIGH ST , SOCIAL ROOM"
20616,3,MADISON,MADISON YMCA FRONT LEFT LOBBY,6:00am-8:00pm,111 KINGS RD
30858,4,MILLBURN,DEERFIELD SCHOOL (H),6:00am-8:00pm,26 TROY LN
51299,6,KINNELON BOROUGH,FAYSON LAKES CLUBHOUSE LARGE ROOM,6:00am-8:00pm,10 CLUBHOUSE TR
55169,7,JERSEY CITY,PS 6 JOTHAM W WAKEMAN SCHOOL,6:00am-8:00pm,100 SAINT PAULS AVE
56237,8,ROCKAWAY TOWNSHIP,DENNIS B O'BRIEN SCHOOL GYM,6:00am-8:00pm,418 MINERAL SPRING DR
57287,9,TENAFLY,PRESBYTERIAN CHURCH AUDITORIUM,6:00am-8:00pm,55 MAGNOLIA AVE
1051,11,MONTCLAIR,BRADFORD SCHOOL (H),6:00am-8:00pm,"87 MOUNT HEBRON RD , (GYM STAGE SIDE)"


In [19]:
#from collections import Counter
#Counter(data_df_1[data_df_1.city=="Seattle"].pollingLocationName)

Counter({'Seattle Union Station': 42})

In [96]:
#data_df_1.voter_id=data_df_1.voter_id-1

In [27]:
my_df_1=pd.read_csv("first_batch_early.csv",header=None).reset_index([0]).merge(data_df_1,how="outer",left_on=['index'],right_on=['voter_id']).iloc[:,[1,3,4,5,6]].rename(columns={0:"homeAddress",1:"county",'city':"pollingCity"})

In [30]:
my_df_1['has_polling_info']=~pd.isnull(my_df_1.pollingAddress)
my_df_1
#my_df_2=my_df_1.drop_duplicates()
#my_df_2

,homeAddress,pollingCity,pollingLocationName,pollingHours,pollingAddress,has_polling_info
0,"80 Old Smalleytown Rd, Warren NJ 07059-5459",WARREN,OUR LADY OF THE MOUNT PARISH CENTER,6:00am-8:00pm,167 MOUNT BETHEL RD,True
1,"625 Iroquois St, Oradell NJ 07649-1219",ORADELL,BOROUGH HALL MULTI PURPOSE RM.2ND FL.,6:00am-8:00pm,355 KINDERKAMACK RD,True
2,"81 High St Unit 11, Orange NJ 07050-1605",ORANGE,SALEM TOWERS (H),6:00am-8:00pm,"98 HIGH ST , SOCIAL ROOM",True
3,"32 Woodland Rd, Madison NJ 07940-2828",MADISON,MADISON YMCA FRONT LEFT LOBBY,6:00am-8:00pm,111 KINGS RD,True
4,"20 Fairfield Dr, Short Hills NJ 07078-1703",MILLBURN,DEERFIELD SCHOOL (H),6:00am-8:00pm,26 TROY LN,True
5,"PO Box 241, New Vernon NJ 07976-0241",NaN,NaN,NaN,NaN,False
6,"19 Birch Rd, Kinnelon NJ 07405-2504",KINNELON BOROUGH,FAYSON LAKES CLUBHOUSE LARGE ROOM,6:00am-8:00pm,10 CLUBHOUSE TR,True
7,"100 Central Ave, Jersey City NJ 07306-2121",JERSEY CITY,PS 6 JOTHAM W WAKEMAN SCHOOL,6:00am-8:00pm,100 SAINT PAULS AVE,True
8,"49 Miller Ave, Rockaway NJ 07866-2230",ROCKAWAY TOWNSHIP,DENNIS B O'BRIEN SCHOOL GYM,6:00am-8:00pm,418 MINERAL SPRING DR,True
9,"27 Stanton Rd, Tenafly NJ 07670-1116",TENAFLY,PRESBYTERIAN CHURCH AUDITORIUM,6:00am-8:00pm,55 MAGNOLIA AVE,True


In [31]:
original=pd.read_csv("civicnation_address_batch1_early_state_clean.csv")
original

,signalvine_id,street,city,zip,state,address,wave,batch
0,5bfadcef-8427-11e6-a6d8-0ad71cd49eb5,80 Old Smalleytown Rd,Warren,7059,NJ,"80 Old Smalleytown Rd, Warren NJ 07059-5459",1,1
1,5bfade6d-8427-11e6-a6d8-0ad71cd49eb5,625 Iroquois St,Oradell,7649,NJ,"625 Iroquois St, Oradell NJ 07649-1219",1,1
2,5bfadee0-8427-11e6-a6d8-0ad71cd49eb5,81 High St Unit 11,Orange,7050,NJ,"81 High St Unit 11, Orange NJ 07050-1605",1,1
3,5bfadf3e-8427-11e6-a6d8-0ad71cd49eb5,32 Woodland Rd,Madison,7940,NJ,"32 Woodland Rd, Madison NJ 07940-2828",1,1
4,5bfadf95-8427-11e6-a6d8-0ad71cd49eb5,20 Fairfield Dr,Short Hills,7078,NJ,"20 Fairfield Dr, Short Hills NJ 07078-1703",1,1
5,5bfadfec-8427-11e6-a6d8-0ad71cd49eb5,PO Box 241,New Vernon,7976,NJ,"PO Box 241, New Vernon NJ 07976-0241",1,1
6,5bfae096-8427-11e6-a6d8-0ad71cd49eb5,19 Birch Rd,Kinnelon,7405,NJ,"19 Birch Rd, Kinnelon NJ 07405-2504",1,1
7,5bfae140-8427-11e6-a6d8-0ad71cd49eb5,100 Central Ave,Jersey City,7306,NJ,"100 Central Ave, Jersey City NJ 07306-2121",1,1
8,5bfae1e8-8427-11e6-a6d8-0ad71cd49eb5,49 Miller Ave,Rockaway,7866,NJ,"49 Miller Ave, Rockaway NJ 07866-2230",1,1
9,5bfae23b-8427-11e6-a6d8-0ad71cd49eb5,27 Stanton Rd,Tenafly,7670,NJ,"27 Stanton Rd, Tenafly NJ 07670-1116",1,1


In [32]:
my_df_1_final=pd.concat([original,my_df_1],join="outer",axis=1)#.drop_duplicates(['homeAddress'])

In [36]:
my_df_1_final=my_df_1_final.drop(['homeAddress'],axis=1)

In [38]:
my_df_1_final.to_csv("batch1_early_v1.csv",index=False)

In [40]:
my_df_1.has_polling_info[my_df_1.has_polling_info==False].to_csv("second_round_index.csv",header=False)

In [42]:
my_df_1_final.address[my_df_1.has_polling_info==False].to_csv("batch1_early_second_round_addresses.csv",header=False,index=False)

In [103]:
wave1_final.drop('homeAddress',axis=1).to_csv("wave1_test_query_result.csv")

In [104]:
wave3_final.drop('homeAddress',axis=1).to_csv("wave3_test_query_result.csv")

In [45]:
my_df_1.to_csv("WA_polling_2.csv",index=False)

In [46]:
df1=pd.read_csv("WA_polling_1.csv")
df2=pd.read_csv("WA_polling_2.csv")

In [48]:
pd.concat([df1,df2],join="outer",axis=0).sort_values(['county','homeAddress'])

,homeAddress,county,pollingCity,pollingLocationName,pollingHours,pollingAddress
69,1000 SR 150 Manson WA,chelan,Wenatchee,Chelan County Auditor - Elections,Monday through Friday 9 a.m. to 5 p.m. Open un...,"350 Orondo Ave, Level 3"
65,1007 N Baker Ave East Wenatchee WA,chelan,Waterville,Douglas County Auditor's Office,Monday through Friday 8:30 a.m. to 4:30 p.m. O...,213 S Rainier St
83,101 Pershing St Wenatchee WA,chelan,Wenatchee,Chelan County Auditor - Elections,Monday through Friday 9 a.m. to 5 p.m. Open un...,"350 Orondo Ave, Level 3"
35,1010 Crawford Ave Wenatchee WA,chelan,Wenatchee,Chelan County Auditor - Elections,Monday through Friday 9 a.m. to 5 p.m. Open un...,"350 Orondo Ave, Level 3"
152,103 Parkhill St Cashmere WA,chelan,Wenatchee,Chelan County Auditor - Elections,Monday through Friday 9 a.m. to 5 p.m. Open un...,"350 Orondo Ave, Level 3"
52,1034 9th St Wenatchee WA,chelan,Wenatchee,Chelan County Auditor - Elections,Monday through Friday 9 a.m. to 5 p.m. Open un...,"350 Orondo Ave, Level 3"
133,10605 Merry Canyon Rd Leavenworth WA,chelan,NaN,NaN,NaN,NaN
73,107 Evergreen Dr Cashmere WA,chelan,Wenatchee,Chelan County Auditor - Elections,Monday through Friday 9 a.m. to 5 p.m. Open un...,"350 Orondo Ave, Level 3"
6,1100 S Chelan Ave Wenatchee WA,chelan,NaN,NaN,NaN,NaN
37,1110 Linwood Ave Wenatchee WA,chelan,Wenatchee,Chelan County Auditor - Elections,Monday through Friday 9 a.m. to 5 p.m. Open un...,"350 Orondo Ave, Level 3"


In [73]:
voter_1=pd.read_csv("sample_addresses_1.csv",header=None).rename(columns={0:"voterAddress"}).reset_index([0]).rename(columns={'index':"voter_id"})
voter_1

,voter_id,voterAddress
0,0,10318 ELYS FORD RD FREDERICKSBURG VA 22407
1,1,9431 COREY DR MANASSAS VA 20110
2,2,2022 WOODGATE ARCH CHESAPEAKE VA 23320
3,3,31 GLACIER WAY STAFFORD VA 22554
4,4,25125 HUMMOCKY TER ALDIE VA 20105
5,5,14146 GREAT SPRING RD SMITHFIELD VA 23430
6,6,224 HOLLY RIDGE RD MOUNT JACKSON VA 22842
7,7,4145 MEW RD CASTLEWOOD VA 24224
8,8,875 VALLEY MILL RD WINCHESTER VA 22602
9,9,590 NORTH FORK RD MARTINSVILLE VA 24112


In [74]:
voter_1.voter_id=voter_1.voter_id+1
voter_1

In [77]:
result_df_1=voter_1.merge(data_df_1,on=["voter_id"],how="outer")

In [80]:
is_available=np.array(["Y"]*50000)
is_available[np.where(pd.isnull(result_df_1.pollingLocationName))[0]]="N"
is_available

array(['Y', 'Y', 'Y', ..., 'Y', 'N', 'Y'], 
      dtype='<U1')

In [82]:
result_df_1["polling_info_available"]=is_available

In [83]:
result_df_1

,voter_id,voterAddress,pollingLocationName,pollingHours,city,pollingAddress,polling_info_available
0,1,10318 ELYS FORD RD FREDERICKSBURG VA 22407,WILDERNESS FIRE & RESCUE BLDG,6:00 AM - 7:00 PM,Spotsylvania,10501 Orange Plank Rd,Y
1,2,9431 COREY DR MANASSAS VA 20110,DEAN ELEMENTARY SCHOOL,6:00 AM - 7:00 PM,Manassas,9601 Prince William St,Y
2,3,2022 WOODGATE ARCH CHESAPEAKE VA 23320,GREENBRIER INTERMEDIATE SCHOOL,6:00 AM - 7:00 PM,Chesapeake,1701 River Birch Run N,Y
3,4,31 GLACIER WAY STAFFORD VA 22554,WIDEWATER ELEMENTARY SCHOOL,6:00 AM - 7:00 PM,Stafford,101 Den Rich Rd,Y
4,5,25125 HUMMOCKY TER ALDIE VA 20105,MERCER MIDDLE SCHOOL,6:00 AM - 7:00 PM,Stone Ridge,42149 Greenstone Dr,Y
5,6,14146 GREAT SPRING RD SMITHFIELD VA 23430,MT TABOR CHURCH OF GOD IN CHRIST,6:00 AM - 7:00 PM,Smithfield,13468 Waterworks Rd,Y
6,7,224 HOLLY RIDGE RD MOUNT JACKSON VA 22842,CONICVILLE FIRE DEPARTMENT,6:00 AM - 7:00 PM,Mount Jackson,763 Conicville Rd,Y
7,8,4145 MEW RD CASTLEWOOD VA 24224,CASTLEWOOD HIGH SCHOOL,6:00 AM - 7:00 PM,Castlewood,304 Blue Devil Cir,Y
8,9,875 VALLEY MILL RD WINCHESTER VA 22602,MILLBROOK HIGH SCHOOL,6:00 AM - 7:00 PM,Winchester,251 First Woods Dr,Y
9,10,590 NORTH FORK RD MARTINSVILLE VA 24112,MVL CLUB,6:00 AM - 7:00 PM,MARTINSVILLE,2485 NORTH FORK ROAD,Y


In [84]:
result_df_1=result_df_1.fillna("")
result_df_1

,voter_id,voterAddress,pollingLocationName,pollingHours,city,pollingAddress,polling_info_available
0,1,10318 ELYS FORD RD FREDERICKSBURG VA 22407,WILDERNESS FIRE & RESCUE BLDG,6:00 AM - 7:00 PM,Spotsylvania,10501 Orange Plank Rd,Y
1,2,9431 COREY DR MANASSAS VA 20110,DEAN ELEMENTARY SCHOOL,6:00 AM - 7:00 PM,Manassas,9601 Prince William St,Y
2,3,2022 WOODGATE ARCH CHESAPEAKE VA 23320,GREENBRIER INTERMEDIATE SCHOOL,6:00 AM - 7:00 PM,Chesapeake,1701 River Birch Run N,Y
3,4,31 GLACIER WAY STAFFORD VA 22554,WIDEWATER ELEMENTARY SCHOOL,6:00 AM - 7:00 PM,Stafford,101 Den Rich Rd,Y
4,5,25125 HUMMOCKY TER ALDIE VA 20105,MERCER MIDDLE SCHOOL,6:00 AM - 7:00 PM,Stone Ridge,42149 Greenstone Dr,Y
5,6,14146 GREAT SPRING RD SMITHFIELD VA 23430,MT TABOR CHURCH OF GOD IN CHRIST,6:00 AM - 7:00 PM,Smithfield,13468 Waterworks Rd,Y
6,7,224 HOLLY RIDGE RD MOUNT JACKSON VA 22842,CONICVILLE FIRE DEPARTMENT,6:00 AM - 7:00 PM,Mount Jackson,763 Conicville Rd,Y
7,8,4145 MEW RD CASTLEWOOD VA 24224,CASTLEWOOD HIGH SCHOOL,6:00 AM - 7:00 PM,Castlewood,304 Blue Devil Cir,Y
8,9,875 VALLEY MILL RD WINCHESTER VA 22602,MILLBROOK HIGH SCHOOL,6:00 AM - 7:00 PM,Winchester,251 First Woods Dr,Y
9,10,590 NORTH FORK RD MARTINSVILLE VA 24112,MVL CLUB,6:00 AM - 7:00 PM,MARTINSVILLE,2485 NORTH FORK ROAD,Y


In [85]:
result_df_1.to_csv("query_results_1.csv",index=False)

In [86]:
data_dict_1={}
for e in json_data_2:
    k=list(e.keys())[0]
    data_dict_1[k]={}
    try:
        address_info=e[k]['pollingLocations'][0]
        try:
            data_dict_1[k]['locationName']=address_info['address']['locationName']
            data_dict_1[k]['city']=address_info['address']['city']
            data_dict_1[k]['line1']=address_info['address']['line1']
            data_dict_1[k]['line2']=address_info['address']['line2']
            data_dict_1[k]['line3']=address_info['address']['line3']
        except KeyError:
            pass
        try:
            data_dict_1[k]['pollingHours']=address_info['pollingHours']
        except KeyError:
            pass
    except KeyError:
        pass

In [87]:
data_df_1=pd.DataFrame.from_dict(data_dict_1,orient="index")
data_df_1=data_df_1.reset_index([0]).rename(columns={'index':'voter_id'})
data_df_1.voter_id=data_df_1.voter_id.astype(int)
data_df_1=data_df_1.sort_values(by=['voter_id']).fillna("")
data_df_1

,voter_id,locationName,line1,pollingHours,city,line2
0,1,KEMPSVILLE MIDDLE SCHOOL,860 Churchill Dr,6:00 AM - 7:00 PM,Virginia Beach,
10153,2,GOOCHLAND COMPANY 5 FIREHOUSE,2710 Fairground Rd,6:00 AM - 7:00 PM,Goochland,
20196,3,BELLEVIEW ELEM SCHOOL,6701 Fort Hunt Rd,6:00 AM - 7:00 PM,Alexandria,
30271,4,BURLINGTON ELEM SCHOOL,6533 Peters Creek Rd,6:00 AM - 7:00 PM,Roanoke,
40309,5,GLADE HILL ELEMENTARY SCHOOL,8081 Old Franklin Tpke,6:00 AM - 7:00 PM,Glade Hill,
41306,6,MARTIN LUTHER KING JR CENTER,683 Oak St,6:00 AM - 7:00 PM,Franklin,
42320,7,CLARKE CO. PUBLIC SCHOOL OFFICE BLDG,"317 W MAIN ST - BERRYVILLE, VA",6:00 AM - 7:00 PM,BERRYVILLE,
43316,8,LAKE DRUMMOND MASONIC LODGE,509 George Washington Hwy N,6:00 AM - 7:00 PM,Chesapeake,
44340,9,AMERICAN LEGION BUILDING,935 ARMORY DR,6:00 AM - 7:00 PM,FRANKLIN,
1,10,Calvary Chapel,1284 Sunset Avenue Ext,6:00 AM - 7:00 PM,Charlottesville,


In [88]:
data_df_1['pollingAddress']=data_df_1.line1+' '+data_df_1.line2
data_df_1['pollingAddress']=data_df_1.pollingAddress.apply(str.strip)

In [89]:
data_df_1=data_df_1.drop(["line1","line2"],axis=1).rename(columns={'locationName':"pollingLocationName"})
data_df_1

,voter_id,pollingLocationName,pollingHours,city,pollingAddress
0,1,KEMPSVILLE MIDDLE SCHOOL,6:00 AM - 7:00 PM,Virginia Beach,860 Churchill Dr
10153,2,GOOCHLAND COMPANY 5 FIREHOUSE,6:00 AM - 7:00 PM,Goochland,2710 Fairground Rd
20196,3,BELLEVIEW ELEM SCHOOL,6:00 AM - 7:00 PM,Alexandria,6701 Fort Hunt Rd
30271,4,BURLINGTON ELEM SCHOOL,6:00 AM - 7:00 PM,Roanoke,6533 Peters Creek Rd
40309,5,GLADE HILL ELEMENTARY SCHOOL,6:00 AM - 7:00 PM,Glade Hill,8081 Old Franklin Tpke
41306,6,MARTIN LUTHER KING JR CENTER,6:00 AM - 7:00 PM,Franklin,683 Oak St
42320,7,CLARKE CO. PUBLIC SCHOOL OFFICE BLDG,6:00 AM - 7:00 PM,BERRYVILLE,"317 W MAIN ST - BERRYVILLE, VA"
43316,8,LAKE DRUMMOND MASONIC LODGE,6:00 AM - 7:00 PM,Chesapeake,509 George Washington Hwy N
44340,9,AMERICAN LEGION BUILDING,6:00 AM - 7:00 PM,FRANKLIN,935 ARMORY DR
1,10,Calvary Chapel,6:00 AM - 7:00 PM,Charlottesville,1284 Sunset Avenue Ext


In [90]:
voter_2=pd.read_csv("sample_addresses_2.csv",header=None).rename(columns={0:"voterAddress"}).reset_index([0]).rename(columns={'index':"voter_id"})
voter_2

,voter_id,voterAddress
0,0,805 WESMERE DR VIRGINIA BEACH VA 23462
1,1,932 LEE RD CROZIER VA 23039
2,2,1303 BELLE VIEW BLVD ALEXANDRIA VA 22307
3,3,6502 CAREFREE LN ROANOKE VA 24019
4,4,5054 OLD FRANKLIN TPKE GLADE HILL VA 24092
5,5,1227 MARINER ST FRANKLIN VA 23851
6,6,7 PAGE ST BERRYVILLE VA 22611
7,7,612 SHELL RD CHESAPEAKE VA 23323
8,8,301 STONEWALL ST FRANKLIN VA 23851
9,9,1629 VICTOR PL CHARLOTTESVILLE VA 22903


In [91]:
voter_2.voter_id=voter_2.voter_id+1
result_df_2=voter_2.merge(data_df_1,on=["voter_id"],how="outer")

In [92]:
is_available=np.array(["Y"]*50000)
is_available[np.where(pd.isnull(result_df_2.pollingLocationName))[0]]="N"
is_available

array(['Y', 'Y', 'Y', ..., 'Y', 'Y', 'Y'], 
      dtype='<U1')

In [93]:
result_df_2=result_df_2.fillna("")
result_df_2["polling_info_available"]=is_available
result_df_2

,voter_id,voterAddress,pollingLocationName,pollingHours,city,pollingAddress,polling_info_available
0,1,805 WESMERE DR VIRGINIA BEACH VA 23462,KEMPSVILLE MIDDLE SCHOOL,6:00 AM - 7:00 PM,Virginia Beach,860 Churchill Dr,Y
1,2,932 LEE RD CROZIER VA 23039,GOOCHLAND COMPANY 5 FIREHOUSE,6:00 AM - 7:00 PM,Goochland,2710 Fairground Rd,Y
2,3,1303 BELLE VIEW BLVD ALEXANDRIA VA 22307,BELLEVIEW ELEM SCHOOL,6:00 AM - 7:00 PM,Alexandria,6701 Fort Hunt Rd,Y
3,4,6502 CAREFREE LN ROANOKE VA 24019,BURLINGTON ELEM SCHOOL,6:00 AM - 7:00 PM,Roanoke,6533 Peters Creek Rd,Y
4,5,5054 OLD FRANKLIN TPKE GLADE HILL VA 24092,GLADE HILL ELEMENTARY SCHOOL,6:00 AM - 7:00 PM,Glade Hill,8081 Old Franklin Tpke,Y
5,6,1227 MARINER ST FRANKLIN VA 23851,MARTIN LUTHER KING JR CENTER,6:00 AM - 7:00 PM,Franklin,683 Oak St,Y
6,7,7 PAGE ST BERRYVILLE VA 22611,CLARKE CO. PUBLIC SCHOOL OFFICE BLDG,6:00 AM - 7:00 PM,BERRYVILLE,"317 W MAIN ST - BERRYVILLE, VA",Y
7,8,612 SHELL RD CHESAPEAKE VA 23323,LAKE DRUMMOND MASONIC LODGE,6:00 AM - 7:00 PM,Chesapeake,509 George Washington Hwy N,Y
8,9,301 STONEWALL ST FRANKLIN VA 23851,AMERICAN LEGION BUILDING,6:00 AM - 7:00 PM,FRANKLIN,935 ARMORY DR,Y
9,10,1629 VICTOR PL CHARLOTTESVILLE VA 22903,Calvary Chapel,6:00 AM - 7:00 PM,Charlottesville,1284 Sunset Avenue Ext,Y


In [95]:
result_df_2.voter_id+=50000
result_df_2

,voter_id,voterAddress,pollingLocationName,pollingHours,city,pollingAddress,polling_info_available
0,50001,805 WESMERE DR VIRGINIA BEACH VA 23462,KEMPSVILLE MIDDLE SCHOOL,6:00 AM - 7:00 PM,Virginia Beach,860 Churchill Dr,Y
1,50002,932 LEE RD CROZIER VA 23039,GOOCHLAND COMPANY 5 FIREHOUSE,6:00 AM - 7:00 PM,Goochland,2710 Fairground Rd,Y
2,50003,1303 BELLE VIEW BLVD ALEXANDRIA VA 22307,BELLEVIEW ELEM SCHOOL,6:00 AM - 7:00 PM,Alexandria,6701 Fort Hunt Rd,Y
3,50004,6502 CAREFREE LN ROANOKE VA 24019,BURLINGTON ELEM SCHOOL,6:00 AM - 7:00 PM,Roanoke,6533 Peters Creek Rd,Y
4,50005,5054 OLD FRANKLIN TPKE GLADE HILL VA 24092,GLADE HILL ELEMENTARY SCHOOL,6:00 AM - 7:00 PM,Glade Hill,8081 Old Franklin Tpke,Y
5,50006,1227 MARINER ST FRANKLIN VA 23851,MARTIN LUTHER KING JR CENTER,6:00 AM - 7:00 PM,Franklin,683 Oak St,Y
6,50007,7 PAGE ST BERRYVILLE VA 22611,CLARKE CO. PUBLIC SCHOOL OFFICE BLDG,6:00 AM - 7:00 PM,BERRYVILLE,"317 W MAIN ST - BERRYVILLE, VA",Y
7,50008,612 SHELL RD CHESAPEAKE VA 23323,LAKE DRUMMOND MASONIC LODGE,6:00 AM - 7:00 PM,Chesapeake,509 George Washington Hwy N,Y
8,50009,301 STONEWALL ST FRANKLIN VA 23851,AMERICAN LEGION BUILDING,6:00 AM - 7:00 PM,FRANKLIN,935 ARMORY DR,Y
9,50010,1629 VICTOR PL CHARLOTTESVILLE VA 22903,Calvary Chapel,6:00 AM - 7:00 PM,Charlottesville,1284 Sunset Avenue Ext,Y


In [98]:
result_df=pd.concat([result_df_1,result_df_2],join="outer",axis=0)

In [100]:
result_df=result_df.rename(columns={"city":"pollingCity"}).iloc[:,[0,1,6,2,4,5,3]]
result_df

,voter_id,voterAddress,polling_info_available,pollingLocationName,pollingCity,pollingAddress,pollingHours
0,1,10318 ELYS FORD RD FREDERICKSBURG VA 22407,Y,WILDERNESS FIRE & RESCUE BLDG,Spotsylvania,10501 Orange Plank Rd,6:00 AM - 7:00 PM
1,2,9431 COREY DR MANASSAS VA 20110,Y,DEAN ELEMENTARY SCHOOL,Manassas,9601 Prince William St,6:00 AM - 7:00 PM
2,3,2022 WOODGATE ARCH CHESAPEAKE VA 23320,Y,GREENBRIER INTERMEDIATE SCHOOL,Chesapeake,1701 River Birch Run N,6:00 AM - 7:00 PM
3,4,31 GLACIER WAY STAFFORD VA 22554,Y,WIDEWATER ELEMENTARY SCHOOL,Stafford,101 Den Rich Rd,6:00 AM - 7:00 PM
4,5,25125 HUMMOCKY TER ALDIE VA 20105,Y,MERCER MIDDLE SCHOOL,Stone Ridge,42149 Greenstone Dr,6:00 AM - 7:00 PM
5,6,14146 GREAT SPRING RD SMITHFIELD VA 23430,Y,MT TABOR CHURCH OF GOD IN CHRIST,Smithfield,13468 Waterworks Rd,6:00 AM - 7:00 PM
6,7,224 HOLLY RIDGE RD MOUNT JACKSON VA 22842,Y,CONICVILLE FIRE DEPARTMENT,Mount Jackson,763 Conicville Rd,6:00 AM - 7:00 PM
7,8,4145 MEW RD CASTLEWOOD VA 24224,Y,CASTLEWOOD HIGH SCHOOL,Castlewood,304 Blue Devil Cir,6:00 AM - 7:00 PM
8,9,875 VALLEY MILL RD WINCHESTER VA 22602,Y,MILLBROOK HIGH SCHOOL,Winchester,251 First Woods Dr,6:00 AM - 7:00 PM
9,10,590 NORTH FORK RD MARTINSVILLE VA 24112,Y,MVL CLUB,MARTINSVILLE,2485 NORTH FORK ROAD,6:00 AM - 7:00 PM


In [101]:
result_df.to_csv("query_results.csv",index=False)